In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import datetime as dt


file_accepted = "accepted_2007_to_2018Q4.csv"

# Read Purchasing File and store into Pandas data frame
acc_data = pd.read_csv(file_accepted)

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
acc_data.head() # 2260701 rows x 151 columns

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#check names of all columns in the data to begin clean up
for col in acc_data.columns:
    print (col)

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
fico_range_low
fico_range_high
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
last_fico_range_high
last_fico_range_low
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
to

# Project Scope:

### We want to predict the loan grade using criteria we chose

1. Loan amount
2. Employement length
3. FICO score (high range)
4. Annual Income
5. Home ownership


In [15]:
#drop all nonessential columns (from 150 columns to 7)
loans_df = acc_data[['loan_amnt', 'grade', 'emp_length', 'application_type', 'annual_inc', 'loan_status', 'fico_range_high']]
loans_df.head()

,loan_amnt,grade,emp_length,application_type,annual_inc,loan_status,fico_range_high
0,3600.0,C,10+ years,Individual,55000.0,Fully Paid,679.0
1,24700.0,C,10+ years,Individual,65000.0,Fully Paid,719.0
2,20000.0,B,10+ years,Joint App,63000.0,Fully Paid,699.0
3,35000.0,C,10+ years,Individual,110000.0,Current,789.0
4,10400.0,F,3 years,Individual,104433.0,Fully Paid,699.0


In [16]:
#drop rows with NaN value left with ~2 million rows
loans_df.dropna(how='any')

,loan_amnt,grade,emp_length,application_type,annual_inc,loan_status,fico_range_high
0,3600.0,C,10+ years,Individual,55000.0,Fully Paid,679.0
1,24700.0,C,10+ years,Individual,65000.0,Fully Paid,719.0
2,20000.0,B,10+ years,Joint App,63000.0,Fully Paid,699.0
3,35000.0,C,10+ years,Individual,110000.0,Current,789.0
4,10400.0,F,3 years,Individual,104433.0,Fully Paid,699.0
...,...,...,...,...,...,...,...
2260694,24000.0,C,7 years,Individual,95000.0,Current,669.0
2260695,24000.0,B,10+ years,Individual,108000.0,Current,699.0
2260696,40000.0,B,9 years,Individual,227000.0,Current,709.0
2260697,24000.0,C,6 years,Individual,110000.0,Charged Off,664.0


In [17]:
# Look at types of loan status data
loans_df['loan_status'].unique()

array(['Fully Paid', 'Current', 'Charged Off', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)', 'Default', nan,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [18]:
# Filter the data with loan_status == fully paid or charged off
loans_filtered=loans_df[(loans_df['loan_status'] == 'Fully Paid') | (loans_df['loan_status'] == 'Charged Off')]

In [21]:
# Check the data
print(loans_filtered['loan_status'].unique())
loans_filtered.head() # 1345310 rows x 7 columns

['Fully Paid' 'Charged Off']


,loan_amnt,grade,emp_length,application_type,annual_inc,loan_status,fico_range_high
0,3600.0,C,10+ years,Individual,55000.0,Fully Paid,679.0
1,24700.0,C,10+ years,Individual,65000.0,Fully Paid,719.0
2,20000.0,B,10+ years,Joint App,63000.0,Fully Paid,699.0
4,10400.0,F,3 years,Individual,104433.0,Fully Paid,699.0
5,11950.0,C,4 years,Individual,34000.0,Fully Paid,694.0


# Extra Data Cleanup

In [23]:
loans_df.to_csv('clean_loans.csv')